In [1]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from empyrical import max_drawdown, alpha, beta
from sklearn.preprocessing import MinMaxScaler
from src.twitter_interface import TwitterInterface
from src.asset_selector import AssetSelector
from src.indicators import Indicators
from util import time_formatter
from src.predictor import Predictor
from sklearn.cluster import KMeans
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py
import pandas as pd
import numpy as np
import configparser
import requests
import twitter
import json
import time
import sys
import os

weeks = 13

backdate = time_formatter(time.time() - (604800 * weeks))
config = configparser.ConfigParser()
try:
    config.read(os.path.relpath("config.ini"))
except FileExistsError as e:
    print("FileExistsError: {}".format(e))
    sys.exit(1)
alpaca_api = tradeapi.REST(
    base_url    = config["alpaca"]["APCA_API_BASE_URL"],
    key_id      = config["alpaca"]["APCA_API_KEY_ID"],
    secret_key  = config["alpaca"]["APCA_API_SECRET_KEY"],
    api_version = config["alpaca"]["VERSION"]
)
twitter_api = twitter.Api(
    config["twitter"]["CONSUMER_KEY"],
    config["twitter"]["CONSUMER_SECRET"],
    config["twitter"]["ACCESS_TOKEN_KEY"],
    config["twitter"]["ACCESS_TOKEN_SECRET"]
)
tq_key = config["tenquant"]["api_key"]

tdf = pd.DataFrame()
ti = TwitterInterface(twitter_api, tdf)
trading_account = alpaca_api.get_account()

selector = AssetSelector(alpaca_api, edgar_token=None)
indicators = Indicators(alpaca_api)

def get_sentiment(text):
    """Given a text block, return a sentiment score based.
    :param text:
    :return:
    """
    text_polarity   = sid.polarity_scores(text)

    if text_polarity["compound"] > 0.50:
        sentiment   = "positive"
    else:
        sentiment   = "negative"
    return sentiment, text_polarity["compound"]

"""
    TODO: move these to asset_selector
"""

def get_losers():
    """Use Polygon endpoint to get a list of top 20 "losers".
    
    :return: 
    """
    print("Losers".center(45))
    print()
    print("Ticker".ljust(10), "Last".ljust(10), "Change".ljust(10), "Pct Gain".ljust(10), "MACD Buy?".ljust(10), "MFI".ljust(10), "Stochastic Oscillator")
    print("{:<30}".format("–" * 45))

    losers = alpaca_api.polygon.gainers_losers("losers")
    
    for symbol in range(len(losers)):
        
        ticker = losers[symbol].ticker
        bars = alpaca_api.get_barset(ticker, "1D", after=backdate)
        dataframe = selector.extract_bar_data(bars, ticker)
        _macd = indicators.get_macd(dataframe)
        macd = _macd["MACD"]
        signal = _macd["SIGNAL"]
        vzo = indicators.get_vzo(dataframe)
        stoch = indicators.get_stoch(dataframe)
        
        try:
            buysignal = macd.iloc[-1] < 0 and min(macd.iloc[-4:-2]) < signal.iloc[-1] and macd.iloc[-1] > signal.iloc[-1]
        except IndexError:
            # Throwing away due to index errors, will handle later
            continue
        
        print(alpaca_api.polygon.gainers_losers("losers")[symbol].ticker.ljust(10),            
            "${:.2f}".format(alpaca_api.polygon.gainers_losers("losers")[symbol].lastTrade["p"]).ljust(10),
            "${:.2f}".format(alpaca_api.polygon.gainers_losers("losers")[symbol].todaysChange).ljust(10),
            "{:.2f}%".format(alpaca_api.polygon.gainers_losers("losers")[symbol].todaysChangePerc).ljust(10),  
        )

def get_gainers():
    """Use Polygon endpoint to get a list of top 20 "gainers".
    
    :return: 
    """
    print("Gainers".center(45))
    print()
    print("Ticker".ljust(10), "Last".ljust(10), "Change".ljust(10), "% Change".ljust(10), "MACD Buy?".ljust(10), "MFI".ljust(10), "VZO".ljust(10), "Stochastic Oscillator")
    print("{:<30}".format("–" * 45))
    
    gainers = alpaca_api.polygon.gainers_losers()
    
    for symbol in range(len(gainers)):
        
        ticker = gainers[symbol].ticker
        bars = alpaca_api.get_barset(ticker, "1D", after=backdate)
        dataframe = selector.extract_bar_data(bars, ticker)
        _macd = indicators.get_macd(dataframe)
        macd = _macd["MACD"]
        signal = _macd["SIGNAL"]
        mfi = indicators.get_mfi(dataframe)
        vzo = indicators.get_vzo(dataframe)
        stoch = indicators.get_stoch(dataframe)
        
        # calculate the MACD buy signal, i.e. if the most recent macd < 0 and < the most recent signal, and the previous MACDs were below the signal
        try:
            buysignal = macd.iloc[-1] < 0 and min(macd.iloc[-4:-2]) <= signal.iloc[-1] and macd.iloc[-1] > signal.iloc[-1]
        except IndexError:
            continue

        # calculate mfi buy signal via bullish 10% crossover 
        try:
            vzo_buysignal = mfi.iloc[-1] > 10 and min(mfi.iloc[-4:-2]) <= 10
        except IndexError:
            continue    
        
        # calculate the VZO buy signal -- look for bullish -40% crossover 
        try:
            mfi_buysignal = vzo.iloc[-1] > -40 and min(vzo.iloc[-4:-2]) <= -40
        except IndexError:
            continue
        
        # calculate stochastic buy signal via bullish 10% crossover 
        try:
            stoch_buysignal = stoch.iloc[-1] > 10 and min(stoch.iloc[-4:-2]) <= 10
        except IndexError:
            continue   

        print(alpaca_api.polygon.gainers_losers()[symbol].ticker.ljust(10),
            "${:.2f}".format(alpaca_api.polygon.gainers_losers()[symbol].lastTrade["p"]).ljust(10),
            "${:.2f}".format(alpaca_api.polygon.gainers_losers()[symbol].todaysChange).ljust(10),
            "{:.2f}%".format(alpaca_api.polygon.gainers_losers()[symbol].todaysChangePerc).ljust(10), 
            str(buysignal).ljust(10),
            str(mfi_buysignal).ljust(10),
            str(vzo_buysignal).ljust(10),
            str(stoch_buysignal).ljust(10),
            "{:.2f}".format(stoch.iloc[-1]).ljust(10)
        )

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/ben/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
pool = 100
get_gainers()
print()
get_losers()
print()
print("Bulls".center(45))
bulls = selector.bullish_candlesticks(64, pool)
# print()
# print("Bears".center(45))
# bears = selector.bearish_candlesticks(64, pool)

                   Gainers                   

Ticker     Last       Change     % Change   MACD Buy?  MFI        VZO        Stochastic Oscillator
–––––––––––––––––––––––––––––––––––––––––––––

                    Losers                   

Ticker     Last       Change     Pct Gain   MACD Buy?  MFI        Stochastic Oscillator
–––––––––––––––––––––––––––––––––––––––––––––

                    Bulls                    
Ticker     Last       Change     % Change   MACD Buy?  MFI        VZO        Stochastic Oscillator Pattern
–––––––––––––––––––––––––––––––––––––––––––––
LBY        $1.68      $0.06      0.01%      True       True       False      True       inverseHammer


sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...


In [2]:
ticker = "LBY"

In [3]:
sid = SentimentIntensityAnalyzer()
stocktwits = requests.post("https://api.stocktwits.com/api/2/streams/symbol/{}.json".format(ticker))
stocktwits = json.loads(stocktwits.text)
stocktwitstext = "\n".join([m["body"] for m in stocktwits["messages"]])
tweets = ti.get_tweets(ticker, "stock")

In [4]:
stsentiment, stscore = get_sentiment(stocktwitstext)
tsentiment, tscore = get_sentiment(tweets)

In [5]:
print("Sentiment score for {}".format(ticker))
print("{:<30}".format("–" * 45))
print("Stocktwits:\t{}\t score: {}\nTwitter:\t{}\t score:\t{}".format(stsentiment, stscore, tsentiment, tscore))
print("Average:\t{}".format((stscore + tscore)/2))

Sentiment score for LBY
–––––––––––––––––––––––––––––––––––––––––––––
Stocktwits:	positive	 score: 0.9967
Twitter:	negative	 score:	0.0
Average:	0.49835


In [6]:
tenquant_data = requests.get("https://api.tenquant.io/data?key={}&ticker={}".format(tq_key, ticker))
tenquant_data = json.loads(tenquant_data.text)

In [7]:
stocktwits

{'response': {'status': 200},
 'symbol': {'id': 6155,
  'symbol': 'LBY',
  'title': 'Libbey Inc.',
  'aliases': [],
  'is_following': False,
  'watchlist_count': 424},
 'cursor': {'more': True, 'since': 184210470, 'max': 181894492},
 'messages': [{'id': 184210470,
   'body': '$LBY well looking like maybe seeing 1.45 coming ?? \n&quot;man, bottom has got to be close i would think?&quot;  \nbuying more if it hit 1.45 hope it gets back to low 3.00 by 2020  \ngood luck to all $$',
   'created_at': '2019-11-19T12:39:54Z',
   'user': {'id': 1558834,
    'username': 'OnlyCash172',
    'name': 'Carl Reiter',
    'avatar_url': 'http://avatars.stocktwits.com/images/default_avatar_thumb.jpg',
    'avatar_url_ssl': 'https://s3.amazonaws.com/st-avatars/images/default_avatar_thumb.jpg',
    'join_date': '2018-07-27',
    'official': False,
    'identity': 'User',
    'classification': [],
    'followers': 7,
    'following': 3,
    'ideas': 697,
    'watchlist_stocks_count': 0,
    'like_count': 324

In [8]:
tweets 

'$LBY:\n\nNew SEC Filing for LBY: Form 8-K (No. 0001437749-19-022993):\n\nhttps://t.co/LvzQKMq1tc\n\n@nahiiila @romane_lby Chui pas stock et on me dit que ça me va bien'

In [9]:
tenquant_data

{'assets': 740892000.0,
 'bookvalue': -10548000.0,
 'comprehensiveincome': -63024000.0,
 'comprehensiveincomeattributabletononcontrollinginterest': 0.0,
 'comprehensiveincomeattributabletoparent': -63024000.0,
 'costofrevenue': 460771000.0,
 'currencycode': 'USD',
 'currentassets': 334791000.0,
 'currentliabilities': 180529000.0,
 'date': '2019-09-30',
 'dividendpayments': 0.0,
 'dividendyield': 0.0,
 'documenttype': '10-Q',
 'duration': 3,
 'equity': -10548000.0,
 'exchangegainslosses': -1111000.0,
 'extraordaryitemsgainloss': 0.0,
 'grossprofit': 115069000.0,
 'incomebeforeequitymethodinvestments': -51766000.0,
 'incomefromcontinuingoperationsaftertax': -51766000.0,
 'incomefromcontinuingoperationsbeforetax': -45255000.0,
 'incomefromequitymethodinvestments': 0.0,
 'incometaxexpensebenefit': 6511000.0,
 'interestanddebtexpense': 17210000.0,
 'liabilities': 751440000.0,
 'liabilitiesandequity': 740892000.0,
 'marketcap': 37334520,
 'netcashflow': 2602000.0,
 'netcashflowsfinancing': 1